In [4]:
import pandas as pd
import numpy as np
import haversine as hv
from sklearn.metrics.pairwise import haversine_distances
from math import radians
import folium
from folium import plugins
from folium.plugins import HeatMap
import seaborn as sns
%matplotlib inline

In [5]:
la = pd.read_csv("la_2019.csv", parse_dates=["start_time","end_time"])

/Users/leazaruchas/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [6]:
# Überblick über Daten
print(la.head())
print(la.tail())

           start_time            end_time  start_station_id  end_station_id  \
0 2019-01-01 00:07:00 2019-01-01 00:14:00              3046            3051   
1 2019-01-01 00:08:00 2019-01-01 00:14:00              3046            3051   
2 2019-01-01 00:18:00 2019-01-01 00:50:00              3030            3075   
3 2019-01-01 00:20:00 2019-01-01 00:50:00              3030            3075   
4 2019-01-01 00:22:00 2019-01-01 00:50:00              3030            3075   

  bike_id user_type start_station_name end_station_name  
0   06468   Walk-up         2nd & Hill   7th & Broadway  
1   12311   Walk-up         2nd & Hill   7th & Broadway  
2   05992   Walk-up         Main & 1st   Broadway & 9th  
3   05860   Walk-up         Main & 1st   Broadway & 9th  
4   06006   Walk-up         Main & 1st   Broadway & 9th  
                start_time            end_time  start_station_id  \
290337 2019-12-31 23:35:51 2020-01-02 17:13:50              4491   
290338 2019-12-31 23:41:52 2019-12-31 23:

In [7]:
print(la.count())
print(la.info())

start_time            290342
end_time              290342
start_station_id      290342
end_station_id        290342
bike_id               290342
user_type             290342
start_station_name    290342
end_station_name      290342
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290342 entries, 0 to 290341
Data columns (total 8 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   start_time          290342 non-null  datetime64[ns]
 1   end_time            290342 non-null  datetime64[ns]
 2   start_station_id    290342 non-null  int64         
 3   end_station_id      290342 non-null  int64         
 4   bike_id             290342 non-null  object        
 5   user_type           290342 non-null  object        
 6   start_station_name  290342 non-null  object        
 7   end_station_name    290342 non-null  object        
dtypes: datetime64[ns](2), int64(2), object(4)
memory usage: 17.7+ MB
None


In [8]:
la.isnull()

,start_time,end_time,start_station_id,end_station_id,bike_id,user_type,start_station_name,end_station_name
0,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...
290337,False,False,False,False,False,False,False,False
290338,False,False,False,False,False,False,False,False
290339,False,False,False,False,False,False,False,False
290340,False,False,False,False,False,False,False,False


In [9]:
# hinzufügen der Dauer der Fahrt
la["duration"] = (la["end_time"] - la["start_time"]).astype("timedelta64[m]")
la["user_type"].size

290342

In [10]:
#remove data with same start and end station, that are shorter or equal to one minutes
print(la[((la["start_station_name"] == la["end_station_name"]) & (la["duration"] <= 1.0))].size)
filtered_data = la[ ~((la["start_station_name"] == la["end_station_name"]) & (la["duration"] <= 1.0)) ]

53325


# Add Cordinates to Data

In [11]:
# new data set with station information (locations)
# https://gbfs.bcycle.com/bcycle_lametro/station_information.json
coordinates = pd.read_csv("station_information.csv")
# drop unnecessary columns
coordinates = coordinates.drop(columns=['ttl', 'data__stations__rental_uris__ios','data__stations__rental_uris__android','version','last_updated'])
coordinates.head(3)

,data__stations__lon,data__stations__lat,data__stations___bcycle_station_type,data__stations__region_id,data__stations__address,data__stations__name,data__stations__station_id
0,-118.25854,34.04850,Kiosk and Station,bcycle_lametro_region_1,Reinstalled 6/15,7th & Flower,bcycle_lametro_3005
1,-118.25667,34.04554,Kiosk and Station,bcycle_lametro_region_1,729 S Olive Street,Olive & 8th,bcycle_lametro_3006
2,-118.25459,34.05048,Kiosk and Station,bcycle_lametro_region_1,557 S 5th Street,5th & Grand,bcycle_lametro_3007


In [12]:
# adding coordinates to stations
coordinates["ziped_coords_start"] = list(zip(coordinates["data__stations__lat"],coordinates["data__stations__lon"]))
la = la.merge(coordinates["ziped_coords_start"].to_frame(),how = "left",left_on=la["start_station_name"],right_on=coordinates["data__stations__name"])
la = la.drop(columns=["key_0"])
# dropping stations without location data
la = la.dropna(axis=0)
la.head(3)

,start_time,end_time,start_station_id,end_station_id,bike_id,user_type,start_station_name,end_station_name,duration,ziped_coords_start
0,2019-01-01 00:07:00,2019-01-01 00:14:00,3046,3051,06468,Walk-up,2nd & Hill,7th & Broadway,7.0,"(34.05287, -118.24748999999998)"
1,2019-01-01 00:08:00,2019-01-01 00:14:00,3046,3051,12311,Walk-up,2nd & Hill,7th & Broadway,6.0,"(34.05287, -118.24748999999998)"
2,2019-01-01 00:18:00,2019-01-01 00:50:00,3030,3075,05992,Walk-up,Main & 1st,Broadway & 9th,32.0,"(34.05194, -118.24353)"


In [13]:
coordinates["ziped_coords_end"] = list(zip(coordinates["data__stations__lat"],coordinates["data__stations__lon"]))
la = la.merge(coordinates["ziped_coords_end"].to_frame(),how = "left",left_on=la["end_station_name"],right_on=coordinates["data__stations__name"])
la = la.drop(columns=["key_0"])

la = la.dropna(axis=0)
la.head(3)

,start_time,end_time,start_station_id,end_station_id,bike_id,user_type,start_station_name,end_station_name,duration,ziped_coords_start,ziped_coords_end
2,2019-01-01 00:18:00,2019-01-01 00:50:00,3030,3075,05992,Walk-up,Main & 1st,Broadway & 9th,32.0,"(34.05194, -118.24353)","(34.04211, -118.25618999999999)"
3,2019-01-01 00:20:00,2019-01-01 00:50:00,3030,3075,05860,Walk-up,Main & 1st,Broadway & 9th,30.0,"(34.05194, -118.24353)","(34.04211, -118.25618999999999)"
4,2019-01-01 00:22:00,2019-01-01 00:50:00,3030,3075,06006,Walk-up,Main & 1st,Broadway & 9th,28.0,"(34.05194, -118.24353)","(34.04211, -118.25618999999999)"


# Make datasets for User Types and map

In [14]:
la["user_type"].unique()

array(['Walk-up', 'Monthly Pass', 'Annual Pass', 'One Day Pass',
       'Flex Pass', 'Testing'], dtype=object)

In [15]:
wu = la[la["user_type"]=="Walk-up"]
print(wu["user_type"].size)
mp = la[la["user_type"]=="Monthly Pass"]
print(mp["user_type"].size)
ap = la[la["user_type"]=="Annual Pass"]
print(ap["user_type"].size)
odp = la[la["user_type"]=="One Day Pass"]
print(odp["user_type"].size)
fp = la[la["user_type"]=="Flex Pass"]
print(fp["user_type"].size)
test = la[la["user_type"]=="Testing"]
print(test["user_type"].size)

59201
144482
17489
9289
270
33


In [16]:
la_cleaned = la

In [17]:
ids = la["bike_id"].unique()
print(ids.size)

4859


In [18]:
la.head()

,start_time,end_time,start_station_id,end_station_id,bike_id,user_type,start_station_name,end_station_name,duration,ziped_coords_start,ziped_coords_end
2,2019-01-01 00:18:00,2019-01-01 00:50:00,3030,3075,05992,Walk-up,Main & 1st,Broadway & 9th,32.0,"(34.05194, -118.24353)","(34.04211, -118.25618999999999)"
3,2019-01-01 00:20:00,2019-01-01 00:50:00,3030,3075,05860,Walk-up,Main & 1st,Broadway & 9th,30.0,"(34.05194, -118.24353)","(34.04211, -118.25618999999999)"
4,2019-01-01 00:22:00,2019-01-01 00:50:00,3030,3075,06006,Walk-up,Main & 1st,Broadway & 9th,28.0,"(34.05194, -118.24353)","(34.04211, -118.25618999999999)"
5,2019-01-01 00:23:00,2019-01-01 00:51:00,3030,3075,06304,Walk-up,Main & 1st,Broadway & 9th,28.0,"(34.05194, -118.24353)","(34.04211, -118.25618999999999)"
6,2019-01-01 00:24:00,2019-01-01 00:51:00,3030,3075,05846,Walk-up,Main & 1st,Broadway & 9th,27.0,"(34.05194, -118.24353)","(34.04211, -118.25618999999999)"


In [19]:
log_map_start = folium.Map(location=(34.052235, -118.243683),tiles='OpenStreetMap',zoom_start=11, control_scale=True, max_zoom=20)
for index, row in wu.iloc[:50].iterrows():
    folium.CircleMarker(radius=5, location=row["ziped_coords_start"], popup=row["start_station_name"], color="green", fill_color="green",weight=2).add_to(log_map_start)
for index, row in mp.iloc[:50].iterrows():
    folium.CircleMarker(radius=5, location=row["ziped_coords_start"], popup=row["start_station_name"], color="blue", fill_color="blue",weight=2).add_to(log_map_start)    
for index, row in odp.iloc[:50].iterrows():
    folium.CircleMarker(radius=5, location=row["ziped_coords_start"], popup=row["start_station_name"], color="red", fill_color="red",weight=2).add_to(log_map_start)
for index, row in fp.iloc[:50].iterrows():
    folium.CircleMarker(radius=5, location=row["ziped_coords_start"], popup=row["start_station_name"], color="yellow", fill_color="yellow",weight=2).add_to(log_map_start)    
for index, row in ap.iloc[:50].iterrows():
    folium.CircleMarker(radius=5, location=row["ziped_coords_start"], popup=row["start_station_name"], color="orange", fill_color="orange",weight=2).add_to(log_map_start)
log_map_start

In [20]:
log_map_end = folium.Map(location=(34.052235, -118.243683),tiles='OpenStreetMap',zoom_start=11, control_scale=True, max_zoom=20)
for index, row in wu.iloc[:50].iterrows():
    folium.CircleMarker(radius=5, location=row["ziped_coords_start"], popup=row["start_station_name"], color="green", fill_color="green",weight=2).add_to(log_map_end)
for index, row in mp.iloc[:50].iterrows():
    folium.CircleMarker(radius=5, location=row["ziped_coords_end"], popup=row["end_station_name"], color="blue", fill_color="blue",weight=2).add_to(log_map_end)    
for index, row in odp.iloc[:50].iterrows():
    folium.CircleMarker(radius=5, location=row["ziped_coords_end"], popup=row["end_station_name"], color="red", fill_color="red",weight=2).add_to(log_map_end)
for index, row in fp.iloc[:50].iterrows():
    folium.CircleMarker(radius=5, location=row["ziped_coords_end"], popup=row["end_station_name"], color="yellow", fill_color="yellow",weight=2).add_to(log_map_end)    
for index, row in ap.iloc[:50].iterrows():
    folium.CircleMarker(radius=5, location=row["ziped_coords_end"], popup=row["end_station_name"], color="orange", fill_color="orange",weight=2).add_to(log_map_end)
log_map_end

# CLUSTERING

In [21]:
# new columns 
#la["Date"] = la["start_time"].apply(lambda x: x.date())
#la["Weekday"]=la["start_time"].apply(lambda x: x.weekday())
#la["Hour"]=la["start_time"].apply(lambda x: x.hour)
#la['Month']= la["start_time"].apply(lambda x: x.month)
#la.head(3)

In [25]:
#distance
la["lat1"], la["lon1"] = list(zip(*la["ziped_coords_start"]))
la["lat2"], la["lon2"] = list(zip(*la["ziped_coords_end"]))
start = la["ziped_coords_start"].tolist()
end = la["ziped_coords_end"].tolist()
start

#la = Dataframe, ziped... =Series -> umwandeln?
#hv.haversine(la["ziped_coords_start"], la["ziped_coords_end"])
#hv.haversine(start, end)
#hv.haversine([la["lat1"], la["lon1"]], [la["lat2"], la["lon2"]])

#pd.DataFrame(haversine_distances(np.radians(la[["lat1","lon1"]]),np.radians(la[["lat2",'lon2"]]))* 6371,index=la.index, columns=la.index)
                               
#la.head()

[(34.05194, -118.24353),
 (34.05194, -118.24353),
 (34.05194, -118.24353),
 (34.05194, -118.24353),
 (34.05194, -118.24353),
 (34.05194, -118.24353),
 (34.05194, -118.24353),
 (34.05194, -118.24353),
 (34.05194, -118.24353),
 (34.04607, -118.23308999999999),
 (34.05287, -118.24748999999998),
 (34.05287, -118.24748999999998),
 (34.04613, -118.25759),
 (34.05088, -118.24825),
 (33.98493, -118.46996000000001),
 (34.04998, -118.24716000000001),
 (33.98493, -118.46996000000001),
 (34.04554, -118.25667),
 (34.04554, -118.25667),
 (34.0484, -118.26095),
 (34.0484, -118.26095),
 (34.0447, -118.25243999999999),
 (34.031890000000004, -118.25018),
 (33.99868, -118.47298),
 (34.0529, -118.24156),
 (34.05287, -118.24748999999998),
 (34.05287, -118.24748999999998),
 (34.0447, -118.25243999999999),
 (33.98434, -118.47155),
 (34.0493, -118.23881000000002),
 (34.03105, -118.26709),
 (34.03105, -118.26709),
 (34.05048, -118.25459),
 (34.04652, -118.23741000000001),
 (34.04652, -118.23741000000001),
 (34

In [ ]:
#City centre for distance calculation
la_centre = [34.052235, -118.243683]
la_centre_rad = [radians(_) for _ in la_centre]

la["start_dist_centre"] = la.apply(lambda x: (haversine_distances([[radians(_) for _ in la_centre], [radians(_) for _ in [x.lat1,x.lon1]]]))[0][1] * 6371000/1000, axis = 1)
la["end_dist_centre"] = la.apply(lambda x: (haversine_distances([[radians(_) for _ in la_centre], [radians(_) for _ in [x.lat2,x.lon2]]]))[0][1] * 6371000/1000, axis = 1)


In [ ]:
la.head()

In [ ]:
la_cleaned
la_cleaned = la_cleaned.drop(columns=["start_time","end_time","start_station_id","end_station_id","bike_id","start_station_name","end_station_name","ziped_coords_start", "ziped_coords_end", "lat1", "lat2", "lon1", "lon2"])

X = la_cleaned.drop("user_type", axis=1)
y = la_cleaned["user_type"]
X.head()


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)
X_scaled_df.head()

In [ ]:
from sklearn.cluster import KMeans
# refit algorithm
two_means = KMeans(n_clusters=6)
two_means.fit(X_scaled_df)

# match records to clusters by calling predict
two_means.predict(X_scaled_df)

In [ ]:
#1. Versuch, siehe plot nächste Zeile
#la_scaled = X_scaled_df
#la_scaled["user_type"] = la["user_type"]


#Kmeans nach clustering mit normalisierten daten zurück auf nicht normalisierte abgebildet
numbers = ["one", "two", "three","four","five","five","six"]
X_scaled_df["clusters"] = two_means.predict(X_scaled_df)
#proof right values match
X_scaled_df["clusters"] = X_scaled_df["clusters"].apply(lambda x: numbers[x])
dist = sns.pairplot(data=X_scaled_df, hue="clusters")

In [ ]:
sns.pairplot(data=la_scaled, hue="user_type")

In [ ]:
#Kmeans nach clustering mit normalisierten daten zurück auf nicht normalisierte abgebildet
numbers = ["one", "two", "three","four","five","five","six"]
y33_plot["clusters"] = two_means.predict(y33_plotter)
#proof right values match
y33_plot["clusters"] = y33_plot["clusters"].apply(lambda x: numbers[x])
dist = sns.pairplot(data=y33_plot, hue="clusters")